## 11. GPT

## План
1. GPT from scratch
2. Transformer++
3. Стратегии декодирования
4. Constrained decoding с json

Датасет возьмём из известного репозитория Карпатого [nanoGPT](https://github.com/karpathy/nanoGPT)

In [1]:
!pip install datasets==2.21.0 transformers nltk

In [2]:
import matplotlib.pyplot as plt

import datasets

In [3]:
text = datasets.load_dataset('karpathy/tiny_shakespeare')['train']['text']

Using the latest cached version of the module from /home/sergey/.cache/huggingface/modules/datasets_modules/datasets/karpathy--tiny_shakespeare/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e (last modified on Thu May 15 15:15:45 2025) since it couldn't be found locally at karpathy/tiny_shakespeare, or remotely on the Hugging Face Hub.


In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sergey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
sentences = sent_tokenize(' '.join(text), language='english')
train_sentences = sentences[:10000]
test_sentences = sentences[10000:]

In [6]:
len(train_sentences), len(test_sentences)

(10000, 993)

In [7]:
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
import torch

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.tokenizer(self.data[idx], return_tensors = 'pt', max_length = self.max_length, truncation=True).input_ids[0]
        input_ids, labels = input_ids[:-1], input_ids[1:]
        attention_mask = torch.ones_like(input_ids, dtype=torch.bool)
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

In [8]:
from torch.nn.utils.rnn import pad_sequence

def data_collator(examples, padding_value=0, max_length=2048):
    def trim_and_pad(seq, batch_first=True, padding_value=0):
        return pad_sequence([s[:max_length] for s in seq], batch_first=batch_first, padding_value=padding_value)

    input_ids = trim_and_pad(
        [example["input_ids"] for example in examples],
        batch_first=True,
        padding_value=padding_value,
    )
    targets = trim_and_pad(
        [example["labels"] for example in examples],
        batch_first=True,
        padding_value=-100,
    )
    attention_mask = trim_and_pad(
        [example["attention_mask"] for example in examples],
        batch_first=True,
        padding_value=0,
    )
    return {
        "input_ids": input_ids,
        "labels": targets,
        "attention_mask": attention_mask,
    }

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.unk_token

In [10]:
from functools import partial

In [11]:
max_length = 32
train_dataset = CustomDataset(train_sentences, tokenizer, max_length=max_length)
test_dataset = CustomDataset(test_sentences, tokenizer, max_length=max_length)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle = True, collate_fn=partial(data_collator, padding_value=tokenizer.pad_token_id, max_length=max_length))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle = False, collate_fn=partial(data_collator, padding_value=tokenizer.pad_token_id, max_length=max_length))

In [12]:
train_dataset[1]

{'input_ids': tensor([3237,   25,  198, 5248,  461,   11, 2740]),
 'labels': tensor([  25,  198, 5248,  461,   11, 2740,   13]),
 'attention_mask': tensor([True, True, True, True, True, True, True])}

In [13]:
batch = next(iter(train_loader))

In [14]:
tokenizer.batch_decode(batch["input_ids"])

['What\nsayest thou to this tune, matter and method<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 'O pity, pity, gentle heaven, pity<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 "They have made worms' meat of me: I have it,\nAnd soundly too: your houses<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>",
 'KING RICHARD III:\nHoyday, a riddle<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

In [15]:
batch["labels"][:5]

tensor([[  198, 16706,   395, 14210,   284,   428, 14009,    11,  2300,   290,
          2446,    30,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [26246,    11, 26246,    11, 10296,  9538,    11, 26246,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [  423,   925, 29174,     6,  6174,   286,   502,    25,   314,   423,
           340,    11,   198,  1870,  2128,   306,  1165,    25,   534,  7777,
             0,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [  371, 20739,  9795,  6711,    25,   198,    39,   726,   820,    11,
           257,   374,  2509,     0,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
 

In [16]:
batch["attention_mask"].sum(1)

tensor([12,  9, 21, 14, 28, 27, 31,  3,  8,  8,  9, 19, 16, 10, 18,  9, 31, 13,
        20,  8, 10, 11,  4, 12, 29, 31, 31, 21, 19, 26,  3, 31, 31, 11, 31, 10,
        31, 17, 19, 20, 11, 31, 11, 19, 18, 16, 18, 17, 26, 11,  6,  4,  4, 14,
         2, 18, 14, 31, 15, 31, 13, 16, 16,  4])

# GPT from scratch

## Positional Encoding
В оригинальном [Трансформере](https://arxiv.org/abs/1706.03762) предложили абсолютные периодические позиционные эмбеддинги:

$$\text{emb}(p, 2i) = \sin(\frac{p}{10000^{2i/d}})$$
$$\text{emb}(p, 2i + 1) = \cos(\frac{p}{10000^{2i/d}})$$
где $p, 2i, 2i+1$ - индексы элемента последовательности, $d$ - длина последовательности.
Они плохо шкалируются на длины за границей тренировочных данных, поэтому появилось много других методов, рассмотрим далее ставший стандартом [RoPE](https://arxiv.org/abs/2104.09864).

In [17]:
import torch
import torch.nn as nn
import math
from tqdm.auto import tqdm
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, seq_len):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]) # (batch, seq_len, d_model)
        return x

## Слой аттеншена, a.k.a. One-Head Scaled Dot-Product Self-Attention

- Каждый элемент выборки $X$ - последовательность фичей $x_i \in \mathbb{R}^D$ переменной длины: $X = \{x_1, x_2, \ldots, x_{n_i}\}$
- То есть $X$ это матрица: $X \in \mathbb{R}^{n_i \times D}$
- Параметры слоя - матрицы фиксированного размера: $W_Q, W_K, W_V \in \mathbb{R}^{D \times d}$
- Вычисление:
    - $Q = X W_Q$ размера $n_i \times d_k$
    - $K = X W_K$ размера $n_i \times d_k$
    - $V = X W_V$ размера $n_i \times d_v$
- $Q, K$ можно нормализовать

## Attention это "мягкий словарь"

- `{key1: value1, key2: value2, ...}`
- **Query** - запрос на поиск среди **Key**
- Пусть $q_i$ и $k_j$ нормализованы: $\|q_i\| = \|k_j\| = 1$
- Тогда $\alpha_{ij}=\langle q_i, k_j \rangle = \cos(\theta)$, где $\theta$ - угол между $q_i$ и $k_j$, можно мерить косинусную схожесть
- Мы хотим померить релевантность всех ключей $k_j, j \in [d]$ запросу $q_i$
- Ответом на $i$-тый запрос будет вернуть линейную комбинацию всех **Value** с найденными скорами релевантности $\alpha_{ij}$

<img src="attention_example.svg" style="width:75%">


$$
\text{Attention}(Q, K, V) = \operatorname{Row-Wise\ Softmax}\left(\dfrac{Q K^\top}{\sqrt{d_k}}\right) V
$$   

*Почему такая нормализация?*
$$
q \sim \mathrm{N}(0, \sigma^2), k \sim \mathrm{N}(0, \sigma^2) \rightarrow \mathbb{V} \left[ \sum_{i=1}^{d_k} q_i k_i\right] = \sum_{i=1}^{d_k} \mathbb{V} \left[  q_i k_i\right] = \sum_{i=1}^{d_k} \mathbb{V}\left[  q_i\right]\mathbb{V} \left[k_i\right]= \sigma^4 d_k
$$
Стандартное отклонение будет пропорционально $\sigma \sqrt{d_k}$, отсюда нормализация.

Если мы не уменьшим дисперсию обратно до $\sigma^2$, то софтмакс по логитам уже насытится до $1$ для одного случайного элемента и $0$ для всех остальных. Градиенты через softmax затухнут, так что мы не сможем выучить параметры должным образом.

На $\sigma$ мы не масштабируем, так как при инициализации весов обычно $\sigma << 1$.

### Multi-Head Attention (MHA)
<img src="mha_img_original.png" style="width:75%">

- Разделяем каждый вектор из $Q, K, V$ на `num_heads` подвекторов с помощью идентичных линейных проекций $h$ раз ($D \mod h = 0$)
- Применяем Attention независимо, соединяем результаты

$$
\begin{align*}
\text{Multihead}(Q, K, V) = \textrm{concat} \left( \text{head}_1, \text{head}_2, \ldots, \text{head}_h \right) W_O,\\
\text{head}_i = \text{Attention}(Q_i, K_i, V_i), i = [h]; W_O \in \mathbb{R}^{h d_v \times d_{out}}
\end{align*}
$$
- **Можно ли написать без цикла?**

In [18]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, qkv_bias = True):
        super().__init__()
        assert d_model % n_heads == 0, "d_model is not divisible by h"
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.query = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.key = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.value = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.out = nn.Linear(d_model, d_model)
        self.scale = self.d_head ** -0.5

    def forward(self, x):
        B, L, D = x.shape
        query = self.query(x).view(B, L, self.n_heads, self.d_head).permute(0, 2, 1, 3)
        key = self.key(x).view(B, L, self.n_heads, self.d_head).permute(0, 2, 1, 3)
        value = self.value(x).view(B, L, self.n_heads, self.d_head).permute(0, 2, 1, 3)

        dots = (query @ key.transpose(-1, -2)) * self.scale

        mask = torch.tril(torch.ones((L, L))).to(x.device)
        dots.masked_fill_(mask == 0, float('-inf'))

        att_scores = dots.softmax(-1)
        att_v = att_scores @ value

        out = att_v.permute(0, 2, 1, 3).contiguous().view(B, L, D)

        out = self.out(out)

        return out


In [19]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, qkv_bias = True):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(d_model)
        self.layer_norm_2 = nn.LayerNorm(d_model)

        self.MHA = MultiHeadAttention(d_model, n_heads, qkv_bias)
        self.MLP = nn.Sequential(nn.Linear(d_model, 4 * d_model), nn.ReLU(), nn.Linear(4 * d_model, d_model))

    def forward(self, x):

        x = self.layer_norm_1(self.MHA(x)) + x
        x = self.layer_norm_2(self.MLP(x)) + x

        return x


In [20]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_blocks, qkv_bias = True, seq_len = 2048):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos = PositionalEncoding(d_model, seq_len)
        self.blocks = nn.Sequential(*[DecoderBlock(d_model, n_heads, qkv_bias) for _ in range(n_blocks)])
        self.final_ln = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = x.to(next(iter(self.parameters())).device)
        emb = self.embedding(x)
        emb = self.pos(emb)

        x = self.blocks(emb)

        out = self.lm_head(self.final_ln(x))

        return out

In [21]:
DEVICE = "cuda:0"
#DEVICE = "mps"
D_MODEL = 512
N_HEADS = 8
VOCAB_SIZE = len(tokenizer)
N_BLOCKS = 3
mha = MultiHeadAttention(D_MODEL, N_HEADS).to(DEVICE)
decoder_block = DecoderBlock(D_MODEL, N_HEADS).to(DEVICE)
model = TransformerDecoder(VOCAB_SIZE, D_MODEL, N_HEADS, N_BLOCKS).to(DEVICE).to(DEVICE)

In [22]:
loss_fnc = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)

/home/sergey/anaconda3/envs/work/lib/python3.8/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/sergey/anaconda3/envs/work/lib/python3.8/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [23]:
from tqdm.auto import tqdm
import numpy as np

In [ ]:
loss_epochs = []
loss_evals = []
for epoch in tqdm(range(10)):
    loss_epoch = []
    model.train()
    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"].to(DEVICE)
        labels_ids = batch["labels"].to(DEVICE)

        preds = model(input_ids)

        optimizer.zero_grad()
        loss = loss_fnc(preds.view(-1, preds.size(-1)), labels_ids.view(-1))
        loss.backward()
        optimizer.step()
        loss_epoch.append(loss.item())

    loss_eval = []
    with torch.no_grad():
        for batch in tqdm(test_loader):
            input_ids = batch["input_ids"].to(DEVICE)
            labels_ids = batch["labels"].to(DEVICE)

            preds = model(input_ids)

            loss = loss_fnc(preds.view(-1, preds.size(-1)), labels_ids.view(-1))
            loss_eval.append(loss.item())

    print(f'Loss_epoch {epoch + 1}: {np.mean(loss_epoch)}')
    print(f'Loss_eval {epoch + 1}: {np.mean(loss_eval)}')
    loss_epochs.append(np.mean(loss_epoch))
    loss_evals.append(np.mean(loss_eval))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Loss_epoch 1: 5.40752817870705
Loss_eval 1: 5.066189527511597


  0%|          | 0/157 [00:00<?, ?it/s]

...

Loss_epoch 3: 3.757310858198032
Loss_eval 3: 4.9725475907325745


  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Loss_epoch 4: 3.4166673292779617
Loss_eval 4: 5.072928071022034


  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
plt.plot(loss_epochs, label="train")
plt.plot(loss_evals, label="eval")
plt.legend()
plt.show()

In [ ]:
batch = train_dataset[3]
input_ids, labels_ids = batch["input_ids"], batch["labels"]

In [ ]:
@torch.no_grad()
def generate_batch(model, ids, is_decode = False, max_length = 20):
    model.eval()
    if type(ids) == list:
        symbols = torch.tensor(ids, device = DEVICE).view(len(ids),1)

    else:
        symbols = torch.tensor(ids, device = DEVICE).view(1,1)

    for _ in range(max_length - 1):

        cur_symbols = model(symbols).argmax(-1)[:, -1][:, None]
        symbols = torch.cat([symbols, cur_symbols], dim = 1)

    if is_decode:
        symbols = tokenizer.batch_decode(symbols, skip_special_tokens=True)
    return symbols

In [ ]:
generate_batch(model, ids = input_ids[0].item(), is_decode=True, max_length= 30)

In [ ]:
tokenizer.decode(labels_ids, skip_special_tokens=True)

# Transformer++

## RMSNorm
Первое архитектурное нововведение - исключение обучаемого `bias` из нормализации. Делает обучение быстрее и стабильнее благодаря своей простоте

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

    def extra_repr(self):
        return f"{tuple(self.weight.shape)}, eps={self.variance_epsilon}"

## RoPE
<img src="Screen_Shot_2021-08-10_at_10.38.41_AM.png" style="width:75%">

RoPE (Rotary Position Embedding) — это способ кодирования позиционной информации в трансформерах, который использует вращательные преобразования для улучшения моделирования отношений между токенами. Вместо прибавления позиционных эмбеддингов к токенам, RoPE вращает их, учитывая относительную позицию токенов в последовательности. Для пары токенов $x_i, x_j$ фазовая разница учитывается напрямую через скалярное произведение, сохраняя относительное расположение токенов:

$$
\operatorname{RoPE}(x) = \begin{bmatrix} x^{(1)} \cos \theta - x^{(2)} \sin \theta \\ x^{(1)} \sin \theta + x^{(2)} \cos \theta \end{bmatrix},
$$


где $\theta=\text{freq}(i)$ определяется в зависимости от позиции $i$ и масштабируется для каждой частоты. Такой подход позволяет трансформеру эффективно обрабатывать как локальные, так и дальние зависимости, улучшая качество на задачах с длинным контекстом.

In [ ]:
from typing import Optional, Tuple
def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    """
    Precompute the frequency tensor for complex exponentials (cis) with given dimensions.

    This function calculates a frequency tensor with complex exponentials using the given dimension 'dim'
    and the end index 'end'. The 'theta' parameter scales the frequencies.
    The returned tensor contains complex values in complex64 data type.

    Args:
        dim (int): Dimension of the frequency tensor.
        end (int): End index for precomputing frequencies.
        theta (float, optional): Scaling factor for frequency computation. Defaults to 10000.0.

    Returns:
        torch.Tensor: Precomputed frequency tensor with complex exponentials.

    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    return freqs_cis


def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    """
    Reshape frequency tensor for broadcasting it with another tensor.

    This function reshapes the frequency tensor to have the same shape as the target tensor 'x'
    for the purpose of broadcasting the frequency tensor during element-wise operations.

    Args:
        freqs_cis (torch.Tensor): Frequency tensor to be reshaped.
        x (torch.Tensor): Target tensor for broadcasting compatibility.

    Returns:
        torch.Tensor: Reshaped frequency tensor.

    Raises:
        AssertionError: If the frequency tensor doesn't match the expected shape.
        AssertionError: If the target tensor 'x' doesn't have the expected number of dimensions.
    """
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1]), f'{freqs_cis.shape} - {(x.shape[1], x.shape[-1])}'
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)


def apply_rotary_emb(
    xq: torch.Tensor,
    xk: torch.Tensor,
    freqs_cis: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Apply rotary embeddings to input tensors using the given frequency tensor.

    This function applies rotary embeddings to the given query 'xq' and key 'xk' tensors using the provided
    frequency tensor 'freqs_cis'. The input tensors are reshaped as complex numbers, and the frequency tensor
    is reshaped for broadcasting compatibility. The resulting tensors contain rotary embeddings and are
    returned as real tensors.

    Args:
        xq (torch.Tensor): Query tensor to apply rotary embeddings.
        xk (torch.Tensor): Key tensor to apply rotary embeddings.
        freqs_cis (torch.Tensor): Precomputed frequency tensor for complex exponentials.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Tuple of modified query tensor and key tensor with rotary embeddings.
    """
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

In [ ]:
class MultiHeadAttentionRoPE(nn.Module):
    def __init__(self, d_model, n_heads, qkv_bias = True):
        super().__init__()
        assert d_model % n_heads == 0, "d_model is not divisible by h"
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.query = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.key = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.value = nn.Linear(d_model, d_model, bias = qkv_bias)
        self.out = nn.Linear(d_model, d_model)
        self.scale = self.d_head ** -0.5

    def forward(self, x):
        B, L, D = x.shape
        query = self.query(x).view(B, L, self.n_heads, self.d_head)
        key = self.key(x).view(B, L, self.n_heads, self.d_head)
        value = self.value(x).view(B, L, self.n_heads, self.d_head).transpose(1,2)
        query, key = apply_rotary_emb(query, key, precompute_freqs_cis(self.d_head, 4096)[:L].to(next(iter(self.query.parameters())).device))
        query = query.transpose(1,2)
        key = key.transpose(1,2)

        dots = (query @ key.transpose(-1, -2)) * self.scale

        mask = torch.tril(torch.ones((L, L))).to(x.device)
        dots.masked_fill_(mask == 0, float('-inf'))

        att_scores = dots.softmax(-1)
        att_v = att_scores @ value

        out = att_v.permute(0, 2, 1, 3).contiguous().view(B, L, D)

        out = self.out(out)

        return out


In [ ]:
class FeedForwardLlama(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.w1 = nn.Linear(d_model, 4 * d_model)
        self.w2 = nn.Linear(4 * d_model, d_model)
        self.w3 = nn.Linear(d_model, 4 * d_model)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

In [ ]:
class DecoderBlockLlama(nn.Module):
    def __init__(self, d_model, n_heads, qkv_bias = True):
        super().__init__()
        self.RMSNorm_1 = RMSNorm(d_model)
        self.RMSNorm_2 = RMSNorm(d_model)

        self.MHA = MultiHeadAttentionRoPE(d_model, n_heads, qkv_bias)
        self.MLP = FeedForwardLlama(d_model)

    def forward(self, x):

        x = self.MHA(self.RMSNorm_1(x)) + x
        x = self.MLP(self.RMSNorm_2(x)) + x

        return x


In [ ]:
class TransformerDecoderLlama(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_blocks, qkv_bias = True, seq_len = 2048):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.blocks = nn.Sequential(*[DecoderBlockLlama(d_model, n_heads, qkv_bias) for _ in range(n_blocks)])
        self.final_ln = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = x.to(next(iter(self.parameters())).device)
        emb = self.embedding(x)

        x = self.blocks(emb)

        out = self.lm_head(self.final_ln(x))

        return out

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
D_MODEL = 256
N_HEADS = 8
VOCAB_SIZE = len(tokenizer)
N_BLOCKS = 3
model_llama = TransformerDecoderLlama(VOCAB_SIZE, D_MODEL, N_HEADS, N_BLOCKS).to(DEVICE)

In [ ]:
loss_fnc = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_llama.parameters(), lr=1e-3)

In [ ]:
model_llama.train()
loss_epochs = []
for epoch in tqdm(range(10)):
    loss_epoch = []
    #for input_ids, labels_ids, data_attention_mask in tqdm(train_loader):
    #    input_ids           = input_ids .to(DEVICE)
    #    labels_ids          = labels_ids.to(DEVICE)
    #    data_attention_mask = data_attention_mask.to(DEVICE)
    for batch in tqdm(train_loader):
        input_ids           = batch["input_ids"     ].to(DEVICE)
        labels_ids          = batch["labels"        ].to(DEVICE)
        data_attention_mask = batch["attention_mask"].to(DEVICE)


        preds = model_llama(input_ids)
        preds = preds[data_attention_mask]
        labels_ids = labels_ids[data_attention_mask]

        optimizer.zero_grad()
        loss = loss_fnc(preds, labels_ids)
        loss.backward()
        optimizer.step()
        loss_epoch.append(loss.item())

    print(f'Loss_epoch {epoch + 1}: {np.mean(loss_epoch)}')
    loss_epochs.append(np.mean(loss_epoch))

In [ ]:
plt.plot(loss_epochs)

In [ ]:
#input_ids, labels_ids, data_attention_mask = train_dataset[2]
batch = train_dataset[2]

input_ids           = batch["input_ids"     ].to(DEVICE)
labels_ids          = batch["labels"        ].to(DEVICE)
data_attention_mask = batch["attention_mask"].to(DEVICE)

In [ ]:
@torch.no_grad()
def generate_batch(model, symbols, is_decode = False, max_length = 20):

    model.eval()
    # if type(ids) == list:
    #     symbols = torch.tensor(ids, device = DEVICE).view(len(ids),1)

    # else:
    #     symbols = torch.tensor(ids, device = DEVICE).view(1,1)

    for _ in range(max_length - 1):

        cur_symbols = model(symbols).argmax(-1)[:, -1][:, None]
        symbols = torch.cat([symbols, cur_symbols], dim = 1)

    if is_decode:
        symbols = tokenizer.batch_decode(symbols, skip_special_tokens=True)
    return symbols

In [ ]:
generate_batch(model_llama, input_ids[:2].view(1, -1).to(DEVICE), is_decode=True, max_length= 28)

In [ ]:
tokenizer.decode(input_ids, skip_special_tokens=True)

In [ ]:
num = 0

for p in model.parameters():
  num+= p.numel()
num

In [ ]:
num = 0

for p in model_llama.parameters():
  num+= p.numel()
num

# Стратегии декодирования

В последние годы растет интерес к языковой генерации благодаря появлению больших языковых моделей на основе трансформеров, обучаемых на миллионах веб-страниц, таких как знаменитая OpenAI [GPT2 model](https://openai.com/blog/better-language-models/). Результаты впечатляют, e.g. [GPT2 on unicorns](https://openai.com/blog/better-language-models/#samples), [XLNet](https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e), [Controlled language with CTRL](https://blog.einstein.ai/introducing-a-conditional-transformer-language-model-for-controllable-generation/). Помимо улучшенной архитектуры transformer и большого объема обучающих данных, важную роль также сыграли улучшенные методы декодирования.

В этом блокноте дается краткий обзор различных стратегий декодирования и, что более важно, показано, как * вы * можете реализовать их с минимальными усилиями, используя популярную библиотеку `transformers`!

Все следующие функциональные возможности могут быть использованы для **авторегрессионной** генерации языка ([тут](http://jalammar.github.io/illustrated-gpt2/) как это делается). Если коротко, *авторегрессионная* генерация языка основана на предположении, что распределение вероятностей последовательности слов может быть разложено на произведение следующих условных вероятностей:
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$


Авторегрессивная генерация языка теперь доступна для `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5`!

Мы проведем экскурсию по наиболее известным в настоящее время методам декодирования, главным образом *Greedy search*, *Beam search*, *Top-K sampling*  и *Top-p sampling*.


In [ ]:
#!pip install transformers -q

In [ ]:
import torch

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [ ]:
# B x seq_len x vocab_size

### **Greedy Search**

Жадный поиск просто выбирает слово с наибольшей вероятностью в качестве следующего слова: $w_t = argmax_{w}P(w | w_{1:t-1})$ на каждом временном шаге $t$. На следующем рисунке показан Greedy Search.

![Greedy Search](greedy_search.png)

Начиная со слова $\text{"The"}$, алгоритм
жадно выбирает следующее слово с наибольшей вероятностью $\text{"nice"}$ и так далее, так что конечная сгенерированная последовательность слов будет $\text{"The", "nice", "woman"}$ с общей вероятностью $0.5 \times 0.4 = 0.2$.

В дальнейшем мы будем генерировать последовательности слов, используя GPT2 в контексте $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$. Давайте посмотрим, как можно использовать жадный поиск в "трансформерах" следующим образом
:

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Хорошо! Мы сгенерировали наш первый короткий текст с помощью GPT2 😊. Сгенерированные слова, следующие контексту, разумны, но модель быстро начинает повторяться! Это очень распространенная проблема при генерации языка в целом и, по-видимому, еще более распространена при grid / beam search'e - проверьте [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) and [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

Однако основным недостатком жадного поиска является то, что он пропускает слова с высокой вероятностью, скрытые за словом с низкой вероятностью, как видно из нашего наброска выше:

Слово $\text{"has"}$ с его высокой условной вероятностью $0,9$ скрыто за словом $\text{"dog"}$, которое имеет только вторую по величине условную вероятность, так что жадный поиск пропускает последовательность слов $\text{"The"}, \text{"dog"}, \text{"has"}$.

К счастью, у нас есть beam search, чтобы решить эту проблему!


### **Beam search**

Поиск по лучу (beam search) снижает риск пропуска скрытых последовательностей слов с высокой вероятностью, сохраняя наиболее вероятные "num_beams" гипотез на каждом временном шаге и в конечном итоге выбирая гипотезу, имеющую в целом наибольшую вероятность. Давайте проиллюстрируем это с помощью `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

На временном шаге $1$, помимо наиболее вероятной гипотезы $\text{"The", "woman"}$, beam search также отслеживает вторую наиболее вероятную гипотезу $\text{"The", "dog"}$. На временном шаге $2$ поиск по лучу обнаруживает, что последовательность слов $\text{"The", "dog", "has"}$ имеет вероятность $0,36$ более высокую, чем $\text{"The", "nice", "woman"}$, которая имеет $0,2$. Отлично, он нашел наиболее вероятную последовательность слов в нашем примере!

Поиск по лучу всегда найдет выходную последовательность с большей вероятностью, чем жадный поиск, но не гарантирует нахождения наиболее вероятного результата.

Давайте посмотрим, как поиск по лучу можно использовать в "трансформерах". Мы устанавливаем `num_beams > 1` и `early_stopping=True`, чтобы генерация завершалась, когда все гипотезы луча достигали токена EOS.

In [ ]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Хотя результат, возможно, более плавный, выход по-прежнему включает повторения одних и тех же последовательностей слов.  
Простое решение состоит в том, чтобы ввести штрафы за *n-граммы* (*также известные как последовательности слов из $n$ слов), как это было введено [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304 ) и [Klein et al. (2017)](https://arxiv.org/abs/1701.02810 ). Наиболее распространенный штраф *n-граммов* гарантирует, что ни один *n-грамм* не появится дважды, вручную устанавливая вероятность следующих слов, которые могли бы создать уже встречавшийся *n-грамм*, равной $0$.

Давайте попробуем это, установив `no_repeat_n_gram_size=2`, чтобы ни один биграм не появлялся дважды:

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=1,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Мило, это выглядит намного лучше! Мы можем видеть, что повторение больше не появляется. Тем не менее, *n-граммовые* штрафы следует использовать с осторожностью. В статье, сгенерированной о городе *Нью-Йорк*, не следует использовать штраф в размере *2-gramms* или название города будет встречаться только один раз во всем тексте!

Еще одной важной особенностью поиска луча является то, что мы можем сравнить верхние лучи после генерации и выбрать сгенерированный луч, который лучше всего соответствует нашей цели.

В `transformers` мы просто устанавливаем параметр `num_return_sequences` на количество лучей с наибольшим количеством баллов, которые должны быть возвращены. Убедитесь, однако, что `num_return_sequences <= num_beams`!

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}\n".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

## Sampling

В своей самой простой форме sampling означает случайный выбор следующего слова $w_t$ в соответствии с его условным распределением вероятностей:

$$w_t \sim P(w|w_{1:t-1})$$

Используя приведенный выше пример, на следующем рисунке визуализируется генерация языка при выборке.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Становится очевидным, что генерация языка с использованием sampling больше не является "детерминированной". Cлово
$\text{"car"}$ выбирается из условного распределения вероятностей $P(w |\text{"The"})$, за которым следует sampling $\text{"drives"}$ из $P(w |\text{"The"}, \text{"car"})$.

В `transformers` устанавливаем `do_sample=True` и деактивируем sampling *Top-K* (подробнее об этом позже) через `top_k=0`. Не стесняйтесь изменять `random_seed`, чтобы поиграть с моделью.

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(1)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Интересно! Текст кажется нормальным, но при ближайшем рассмотрении оказывается, что он не очень связный. Слова *new hand sense* и *local batte harness* очень странные и звучат не так, как будто они были написаны человеком. Это большая проблема при выборке последовательностей слов: модели часто генерируют бессвязную тарабарщину, *ср.* [Ари Хольцман и др. (2019)](https://arxiv.org/abs/1904.09751 ).

Хитрость заключается в том, чтобы сделать распределение $P(w|w_{1:t-1})$ более четким (увеличивая вероятность слов с высокой вероятностью и уменьшая вероятность слов с низкой вероятностью), понизив так называемую "температуру" [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max ).

$q = \frac{exp(z_i / T)}{\sum_j exp(z_j / T)}$

Иллюстрация применения температуры к нашему примеру, приведенному выше, может выглядеть следующим образом.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

Условное распределение следующего слова на шаге $t = 1$ становится намного более четким, практически не оставляя шансов для выбора слова $\text{"car"}$.


In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

## **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf ) ввел простую, но очень мощную схему семплирования, называемую выборкой ***Top-K***. В выборке *Top-K* фильтруются *K* наиболее вероятных следующих слов, и масса вероятности перераспределяется только между этими *K* следующими словами.
GPT2 принял эту схему семплирования, что стало одной из причин его успеха.

Мы расширили диапазон слов, используемых для обоих этапов семплирования в приведенном выше примере, с 3 слов до 10 слов, чтобы лучше проиллюстрировать выборку *Top-K*.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Установив $K = 6$, на обоих этапах семплирования мы ограничиваем наш пул выборок 6 словами. В то время как 6 наиболее вероятных слов, определенных как $V_ {\text{top-K}}$, охватывают только *около* двух третей всей массы вероятности на первом шаге, они включают почти всю массу вероятности на втором шаге. Тем не менее, мы видим, что он успешно устраняет довольно странных кандидатов $\text{"not", "the", "small", "told"}$
на втором этапе отбора проб.


Давайте посмотрим, как *Top-K* можно использовать в библиотеке, установив `top_k=50`:


In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.9
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Совсем неплохо! Этот текст, пожалуй, самый "человечный" на сегодняшний день.
Однако одна проблема с выборкой *Top-K* заключается в том, что она динамически не адаптирует количество слов, которые отфильтровываются из распределения вероятности следующего слова $P (w |w_ {1:t-1})$.
Это может быть проблематично, поскольку некоторые слова могут быть выбраны из очень четкого распределения (распределение справа на графике выше), в то время как другие - из гораздо более плоского распределения (распределение слева на графике выше).

На шаге $t=1$, *Top-K* исключает возможность
примеров $\text{"people", "big", "house", "cat"}$, которые кажутся разумными кандидатами. С другой стороны, на шаге $t= 2$ метод включает, возможно, неправильно подобранные слова $\text{"down", "a"}$ в выборку слов. Таким образом, ограничение пула выборок фиксированным размером *K* может привести к тому, что модель будет выдавать тарабарщину для четких распределений и ограничит творческий потенциал модели для плоского распределения.
Эта интуиция привела [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751 ) для создания ***Top-p*** - или ***nucleus***-sampling.

## **Sampling Top-p (nucleus)**

Вместо выборки только из наиболее вероятных *K* слов, в выборке *Top-p* выбирается из наименьшего возможного набора слов, совокупная вероятность которых превышает вероятность *p*. Затем масса вероятности перераспределяется между этим набором слов. Таким образом, размер набора слов (количество слов в наборе) может динамически увеличиваться и уменьшаться в соответствии с распределением вероятности следующего слова. Ладно, это было очень многословно, давайте визуализируем.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Установив $p=0.92$, выборка *Top-p* выбирает *минимальное* количество слов, которое в совокупности должно превышать $p=92\%$ от массы вероятности, определяемой как $V_{\text{top-p}}$. В первом примере это включало 9 наиболее вероятных слов, тогда как во втором примере нужно выбрать только 3 лучших слова, чтобы их вероятностная масса превысила 92%. На самом деле все очень просто! Можно видеть, что стратегия сохраняет широкий диапазон слов, где следующее слово, возможно, менее предсказуемо, *например* $P(w | \text{"The"})$, и только несколько слов, когда следующее слово кажется более предсказуемым, *например* $P(w | \text{"The", "car"})$.

Мы активируем выборку *Top-p*, установив `0 < top_p < 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(3)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.8,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Отлично, звучит так, словно это мог бы написать человек. Ну, может быть, еще не совсем.

Хотя в теории *Top-p* кажется более элегантным, чем *Top-K*, оба метода хорошо работают на практике. *Top-p* также можно использовать в сочетании с *Top-K*, что позволяет избежать слов с очень низким скором, обеспечивая при этом некоторый динамический выбор.

Наконец, чтобы получить несколько независимо отобранных выходных данных, мы можем установить параметр `num_return_sequences > 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.random.manual_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

## Constrained generation

Для лучшей читаемости мы будем использовать end-of-line в качестве токена EOS, вместо того, чтобы просто всегда генерировать 50 токенов.

In [ ]:
END_OF_LINE = tokenizer('\n').input_ids[0]
print(END_OF_LINE)

Поиск по базовому лучу выдает очень похожие предложения; все они содержат "not sure" или "don't think".

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Что произойдет, если мы запретим модели использовать эти фразы?



In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think', 'thundersnatch'], add_prefix_space=True)['input_ids'],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

In [ ]:
tokenizer(['sure', ' sure', ' I am not sure'])['input_ids']

Мы видим, что смысл этих текстов сильно изменился – но каким-то непредсказуемым образом.

Можем ли мы заставить модель написать текст с участием "cat"?

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids=[tokenizer(['cat'], add_prefix_space=True, add_special_tokens=False).input_ids],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Уточнение: **force_words_ids** - это список ограничений. Каждое ограничение представляет собой список выражений, таких, что по крайней мере одно выражение должно быть включено в сгенерированный текст. И каждое выражение - это просто список токенов.

Смотрите обсуждение в [the HF pull request](https://github.com/huggingface/transformers/issues/14081 ), или прочтите статью "[Guided Generation of Cause and Effect](https://www.ijcai.org/proceedings/2020/0502.pdf )" Ли и др., где был предложен алгоритм.  

Чтобы оценить силу этих ограничений, давайте заставим модель включить мышь (или даже много мышей) в текст. Мы также можем ослабить ограничение "кошка", разрешив вместо него использовать слова "кошки", "котенок" или "кошачий".

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True,
    eos_token_id=END_OF_LINE,
    bad_words_ids=tokenizer(['sure', 'think'], add_prefix_space=True)['input_ids'],
    force_words_ids = [
        tokenizer(['cat', 'cats', 'kitten', 'feline', 'Cat', 'Cats'], add_prefix_space=True, add_special_tokens=False).input_ids,
        tokenizer(['mouse', 'mice'], add_prefix_space=True, add_special_tokens=False).input_ids,
    ],
)

print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

## **Заключение**

Методы декодирования *ad-hoc*, выборка *top-p* и *top-K*, по-видимому, обеспечивают более плавный текст, чем традиционный *жадный* и *лучевой* поиск при генерации открытого языка.
Однако в последнее время появилось больше доказательств того, что очевидные недостатки *жадного* и *лучевого* поиска - в основном генерирующие повторяющиеся последовательности слов - вызваны моделью (особенно способом обучения модели), а не методом декодирования,  [Уэллек и др. (2019)](https://arxiv.org/pdf/1908.04319.pdf ). Кроме того, как показано в [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492 ), похоже, что выборки *top-K* и *top-p* также страдают от генерации повторяющихся последовательностей слов.

В [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf ), авторы показывают, что, согласно человеческим оценкам, лучевой поиск может генерировать более плавный текст, чем выборка *Top-p*, при адаптации цели обучения модели.

## **Приложение**

Есть пара дополнительных параметров для метода `generate`, которые не были упомянуты выше. Мы кратко объясним их здесь!

- `min_length` можно использовать, чтобы заставить модель не выдавать токен EOS (= не заканчивать предложение) до достижения `min_length`. Это довольно часто используется при суммаризации, но может быть полезно в целом, если пользователь хочет получить более длинные выходные данные.
- `repetition_penalty` может использоваться для "наказания" слов, которые уже были сгенерированы или принадлежат контексту. Впервые он был представлен [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858 ) и также используется в цели обучения в [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf ). Это может быть довольно эффективно для предотвращения повторений, но, по-видимому, очень чувствительно к различным моделям и вариантам использования, *например* смотрите это [обсуждение](https://github.com/huggingface/transformers/pull/2303 ) на Github.

- `attention_mask` можно использовать для маскировки дополненных токенов
- `pad_token_id`, `bos_token_id`, `eos_token_id`: Если в модели по умолчанию нет этих токенов, пользователь может вручную выбрать другие идентификаторы токенов для их представления.


In [ ]:
!pip install vllm lm-format-enforcer triton -q

We load the model, as is normally done with vLLM

In [ ]:
import torch

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = LLM(model=model_id, dtype=torch.float16)


In [ ]:
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

## Скомбинируем CharacterLevelParser с vLLM

In [ ]:
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.vllm import build_vllm_logits_processor, build_vllm_token_enforcer_tokenizer_data
from typing import Union, List, Optional
from vllm import SamplingParams

DEFAULT_MAX_NEW_TOKENS = 100

ListOrStrList = Union[str, List[str]]

tokenizer_data = build_vllm_token_enforcer_tokenizer_data(llm)

def vllm_with_character_level_parser(prompt: ListOrStrList, parser: Optional[CharacterLevelParser] = None) -> ListOrStrList:
    sampling_params = SamplingParams()
    sampling_params.max_tokens = DEFAULT_MAX_NEW_TOKENS
    if parser:
        logits_processor = build_vllm_logits_processor(tokenizer_data, parser)
        sampling_params.logits_processors = [logits_processor]
    results = llm.generate(prompt, sampling_params=sampling_params)
    if isinstance(prompt, str):
        return results[0].outputs[0].text
    else:
        return [result.outputs[0].text for result in results]


## vLLM + JSON Use case

Теперь мы продемонстрируем использование ``JsonSchemaParser``. Мы создаем Pydantic model, генерируем на ее основе схему и используем ее для costrained decoding.
Выходные данные всегда будут иметь формат, который может быть разобран парсером.

In [ ]:
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel


class AnswerFormat(BaseModel):
    first_name: str
    last_name: str
    year_of_birth: int
    num_seasons_in_nba: int

question = 'Please give me information about Michael Jordan. You MUST answer using the following json schema: '
question_with_schema = f'{question}{AnswerFormat.model_json_schema()}'

sampling_params = SamplingParams(temperature=0.6, top_p=0.92, max_tokens=100)

messages = [
    {"role": "user", "content": question_with_schema},
]

prompts = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

display_header("Prompt:")
display_content(prompts)

display_header("Answer, With json schema enforcing:")

result = vllm_with_character_level_parser(prompts, JsonSchemaParser(AnswerFormat.model_json_schema()))
display_content(result)

display_header("Answer, Without json schema enforcing:")
result = vllm_with_character_level_parser(prompts, None)
display_content(result)


As you can see, the enforced output matches the required schema, while the unenforced does not. We have successfully integrated with vLLM!

## Batching example

Now we demonstrate that the model can be used to generate text in batches. This is useful for generating text in parallel, which is much faster than generating text sequentially.

In [ ]:
from time import time

players = ['Michael Jordan', 'Tim Duncan', 'Larry Bird', 'Magic Johnson', 'Patrick Ewing',
           'Hakeem Olajuwan', 'Nate Archibald', 'Charles Barkley', 'Bob Cousy', 'Clyde Drexler',
           'Julius Erving', 'John Havlicek', 'Elvin Hayes', 'Jerry Lucas', 'Moses Malone',
           'George Mikan', 'Bob Pettit', 'Oscar Robertson', 'Bill Russell', 'Dolph Schayes']
prompts = []
for player in players:
    question = f'Please give me information about {player}. You MUST answer using the following json schema: '
    question_with_schema = f'{question}{AnswerFormat.schema_json()}'

    messages = [
        {"role": "user", "content": question_with_schema},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    prompts.append(prompt)

start = time()
one_player_result = vllm_with_character_level_parser(prompts[0], JsonSchemaParser(AnswerFormat.schema()))
end = time()
print(f'Time taken for 1 player: {end - start}s')
display_content(one_player_result)

start = time()
all_results = vllm_with_character_level_parser(prompts[1:], JsonSchemaParser(AnswerFormat.schema()))
end = time()
print(f'Time taken for {len(prompts)-1} players: {end - start}. Time per player: {(end - start)/(len(prompts)-1)}')
display_content(all_results)

In [ ]:
print(all_results)

# Регулярное выражение + пример анализа

Теперь мы покажем две дополнительные возможности: Поддержка регулярных выражений и анализ помех.

Код здесь более низкоуровневый, так как нам нужен экземпляр `logits_processor`, чтобы не вызывать вспомогательную функцию `vllm_with_character_level_parser`, которую мы создали ранее в этом блокноте.

Анализ вмешательства позволяет нам увидеть, насколько сильно пришлось действовать форматировщику и какова была бы вероятность появления выбранных лексем, если бы форматировщик не вмешивался. Это может помочь вам улучшить качество результатов, улучшив подсказки и моделирование, чтобы уменьшить редактирование декодируемой последовательности. Как правило, чем меньше вмешательства, тем лучше.

In [ ]:
from lmformatenforcer.regexparser import RegexParser
import pandas as pd

date_regex = r'(0?[1-9]|1[0-2])\/(0?[1-9]|1\d|2\d|3[01])\/(19|20)\d{2}'
answer_regex = ' In mm/dd/yyyy format, Michael Jordan was born in ' + date_regex
parser = RegexParser(answer_regex)

question = 'When was Michael Jordan Born? Please answer in mm/dd/yyyy format.'
messages = [
    {"role": "user", "content": question},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
display_header("Prompt:")
display_content(prompt)

# Note the analyze=True flag, which is will create an analyzer in the processor
logits_processor = build_vllm_logits_processor(tokenizer_data, parser, analyze=True)

sampling_params = SamplingParams(max_tokens=200, logits_processors=[logits_processor])
results = llm.generate(prompt, sampling_params=sampling_params)

text = results[0].outputs[0].text
display_header("Answer:")
display_content(text)

display_header("Analyzer Results:")
report_dict = logits_processor.analyzer.generate_report_dict(results[0].outputs[0].token_ids)
enforced_scores = pd.DataFrame(report_dict)
# Setting some display options for readability
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 999)
pd.set_option('display.float_format', ' {:,.5f}'.format)
display(enforced_scores)


The timesteps in which `generated_score < leading_score` are those in which the format enforcer had to intervene. Consider using this during development to fine tune your prompts for better consistency.

This method also works for JSON Schema mode, of course.